In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# --- Step 1: Load Data ---
file_path = r"C:\Users\qwert\Desktop\data.csv" 
df = pd.read_csv(file_path)

print("✅ Dataset Loaded")
display(df.head())

# --- Step 2: Preprocess Data ---
le = LabelEncoder()

# Clean "Your Age"
df["Your Age"] = df["Your Age"].astype(str).str.strip()           
df["Your Age"] = df["Your Age"].str.replace(r"\D", "", regex=True) 
df["Your Age"] = pd.to_numeric(df["Your Age"], errors="coerce")    
df["Your Age"] = df["Your Age"].fillna(df["Your Age"].median())    
# Encode categorical columns except Age
for col in df.columns:
    if col not in ["Your Age", "Your Previous Semester Grade Range"] and df[col].dtype == "object":
        df[col] = df[col].fillna("Unknown")
        df[col] = le.fit_transform(df[col].astype(str))
# --- Step 3: Define Features & Target ---
stress_map = {
    0: "High",
    1: "Low",
    2: "Normal",
    3: "Unknown",
    4: "Very high",
    5: "Very Low"
}

df["What is your current level of academic stress?"] = df["What is your current level of academic stress?"].map(stress_map)
target = "What is your current level of academic stress?"

# Encode target separately
stress_encoder = LabelEncoder()
y = stress_encoder.fit_transform(df[target])


X = df.drop(columns=[target, "Your Previous Semester Grade Range"])  # drop text column

# --- Step 4: Split Dataset ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 5: Scale Features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Step 6: Train Random Forest ---
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train_scaled, y_train)

# --- Step 7: Predictions & Evaluation ---
y_pred = model.predict(X_test_scaled)
print("\n🎯 Model Accuracy:", accuracy_score(y_test, y_pred))


# --- Step 8: Academic Risk Calculation ---
def grade_to_score(grade_str):
    """Convert grade range into numeric midpoint."""
    mapping = {
        "0–40 (Weak)": 20,
        "41–60 (Average)": 50,
        "61–80 (Good)": 70,
        "81–100 (Excellent)": 90
    }
    return mapping.get(grade_str, 50)  # default = 50 if missing

df["Grade_Score"] = df["Your Previous Semester Grade Range"].map(grade_to_score)

def academic_risk(final_grade, stress_label):
    if final_grade < 50 or stress_label in ["High", "Very high"]:
        return "At Risk"
    elif final_grade < 75:
        return "Average"
    else:
        return "Excellent"

# Add predictions
df["Predicted_Stress"] = stress_encoder.inverse_transform(
    model.predict(scaler.transform(X))
)
df["Academic_Risk"] = df.apply(lambda r: academic_risk(r["Grade_Score"], r["Predicted_Stress"]), axis=1)

# --- Step 9: Personalized Recommendations ---
def recommend(row):
    tips = []

    stress_label = row["Predicted_Stress"]

    if str(stress_label).lower() in ["high", "very high"]:
        tips.append("✅ Sleep 7–8h, take short breaks, practice relaxation")
    if row["Grade_Score"] < 50:
        tips.append("📘 Focus on improving study routine")
    if row["How many classes have you missed this semester?"] > 5:
        tips.append("📅 Reduce class absences")
    if row["How many hours do you usually sleep per night?"] < 5:
        tips.append("😴 Improve sleep habits")
    if row["How many times per week do you exercise?"] < 2:
        tips.append("💪 Add light exercise to reduce stress")

    return "; ".join(tips) if tips else "👍 Keep up the good work!"

df["Recommendations"] = df.apply(recommend, axis=1)
pd.set_option('display.max_colwidth', None)
# --- Step 10: Show Sample Results ---
print("\n📌 Sample Student Predictions:")
display(
    df[[
        "Your Age", "Gender", "Year of study", "Your Major",
        "Your Previous Semester Grade Range",
        "Predicted_Stress", "Academic_Risk", "Recommendations"
    ]].rename(columns={
        "Your Age": "Age",
        "Gender": "Sex",
        "Year of study": "Year",
        "Your Major": "Major",
        "Your Previous Semester Grade Range": "Previous Grade",
        "Predicted_Stress": "Stress Prediction",
        "Academic_Risk": "Performance Level",
        "Recommendations": "Advice"
    }).sample(n=15, random_state=1)
)


✅ Dataset Loaded


,အချိန်ဖော်ပြချက်,Your Age,Gender,Year of study,Your Major,Your Previous Semester Grade Range,How many subjects have you failed in past semesters?,How many classes have you missed this semester?,How often do you participate in class?,How much do you engage in group projects or assignments?,How do you rate your overall health?,How many hours do you usually sleep per night?,How much free time do you have after school?,How often do you go out with friends?,How many times per week do you exercise?,What is your current level of academic stress?,What are the main sources of your stress?,What methods do you usually use to deal with stress?,What is your biggest challenge as a university student?,What resources would help reduce your stress?
0,၂၀၂၅/၀၉/၀၆ ၇:၄၄:၂၆ ညနေ GMT+၆:၃၀,22,Female,4.0,CS,61–80 (Good),NaN,NaN,Often,Always,Normal,6–7 hours,Fair amount,Often,1–2 times,Normal,Exams & assignments,Relaxation,Doing project,Eating foods
1,၂၀၂၅/၀၉/၀၆ ၇:၄၄:၃၆ ညနေ GMT+၆:၃၀,22,Female,4.0,CS,61–80 (Good),NaN,10 or more classes,Often,Always,Good,8–9 hours,Fair amount,Often,1–2 times,Normal,Exams & assignments,Hobbies,Exam stress,Music
2,၂၀၂၅/၀၉/၀၆ ၇:၄၈:၂၆ ညနေ GMT+၆:၃၀,23,Female,4.0,CS,61–80 (Good),NaN,10 or more classes,Sometimes,Often,Good,8–9 hours,Fair amount,Rarely,1–2 times,High,Exams & assignments;Social life,Relaxation,Presenting confidently in front of audience\n,Access to sports or relaxation activities
3,၂၀၂၅/၀၉/၀၆ ၇:၅၂:၀၀ ညနေ GMT+၆:၃၀,23,Female,4.0,CS,61–80 (Good),NaN,10 or more classes,Sometimes,Always,Good,6–7 hours,Little,Sometimes,Never,Very high,Exams & assignments;Attendance & participation;Social life,Hobbies;Games,Exams and projects,"Food, games"
4,၂၀၂၅/၀၉/၀၆ ၇:၅၂:၂၆ ညနေ GMT+၆:၃၀,24,Female,4.0,CS,61–80 (Good),NaN,NaN,Sometimes,Often,Excellent,8–9 hours,Fair amount,Sometimes,3–4 times,High,Exams & assignments,Relaxation,Exam,Relax in peace



🎯 Model Accuracy: 0.6538461538461539

📌 Sample Student Predictions:


,Age,Sex,Year,Major,Previous Grade,Stress Prediction,Performance Level,Advice
122,22.0,0,1.0,1,41–60 (Average),Normal,Average,😴 Improve sleep habits; 💪 Add light exercise to reduce stress
104,20.0,0,1.0,1,81–100 (Excellent),Normal,Excellent,😴 Improve sleep habits; 💪 Add light exercise to reduce stress
55,19.0,0,1.0,1,61–80 (Good),Normal,Average,😴 Improve sleep habits
56,19.0,0,1.0,1,61–80 (Good),High,At Risk,"✅ Sleep 7–8h, take short breaks, practice relaxation; 😴 Improve sleep habits; 💪 Add light exercise to reduce stress"
33,19.0,1,1.0,1,61–80 (Good),Normal,Average,😴 Improve sleep habits
119,19.0,0,1.0,1,41–60 (Average),Normal,Average,😴 Improve sleep habits; 💪 Add light exercise to reduce stress
123,19.0,1,1.0,1,NaN,Low,Average,😴 Improve sleep habits
35,23.0,0,4.0,0,61–80 (Good),Normal,Average,😴 Improve sleep habits; 💪 Add light exercise to reduce stress
31,22.0,0,4.0,0,61–80 (Good),Normal,Average,😴 Improve sleep habits; 💪 Add light exercise to reduce stress
51,22.0,0,4.0,0,0–40 (Weak),High,At Risk,"✅ Sleep 7–8h, take short breaks, practice relaxation; 📘 Focus on improving study routine; 😴 Improve sleep habits"
